In [3]:
import pandas as pd
import numpy as np

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.chrome.options import Options
from datetime import datetime, timedelta
from oceanoobsbrasil.db import GetData
from oceanoobsbrasil.utils import *
from oceanoobsbrasil.buoys.aqualink import AqualinkBuoy

import os
from dotenv import load_dotenv

from bs4 import BeautifulSoup
import re
import chromedriver_binary
import time

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [34]:
a = AqualinkBuoy()

In [35]:
a.stations

,id,name,lat,lon,data_type,institution,url,flag
0,1217,Ilha do Farol,-23.001,-42.007,buoy,aqualink,1187,True
1,1230,Fernando de Noronha,-3.859,-32.449,buoy,aqualink,1186,True


In [36]:
a.get()

https://aqualink.org/sites/1187
WINDSPEED4.3km/hDIRECTION125°WAVESHEIGHT0.3mPERIOD5sDIRECTION320°Last data received 20 min. agoUpdated hourlyLIVE
['4.3', '125', '0.3', '5', '320', '20', 'min.']
       wspd wdir swvht tp wvdir   sst            date_time station_id
0  2.321815  125   0.3  5   320  24.9  2024-01-21 18:00:00       1217
https://aqualink.org/sites/1186
WINDSPEED11.5km/hDIRECTION17°WAVESHEIGHT1.3mPERIOD8sDIRECTION343°Last data received 29 min. agoUpdated hourlyLIVE
['11.5', '17', '1.3', '8', '343', '29', 'min.']
       wspd wdir swvht tp wvdir   sst            date_time station_id
0  6.209506   17   1.3  8   343  28.8  2024-01-21 18:00:00       1230
driver is running
chrome is still running, we can quit
driver has died


In [16]:
a.driver = webdriver.Chrome(options=a.options)
a.driver.get('https://aqualink.org/sites/1187')

In [17]:
soup=BeautifulSoup(a.driver.page_source, 'html.parser')

In [18]:
elements = soup.find_all(attrs={'class': 'MuiCard-root'})

In [28]:
elements[3].text

'HEAT STRESS ALERT LEVELLast data received 6 days agoUpdated dailyNOAA CRW'

In [29]:
'WIND' in elements[4].text

True

In [30]:
text = elements[4].text

In [31]:
text

'WINDSPEED4.3km/hDIRECTION120°WAVESHEIGHT0.3mPERIOD5sDIRECTION324°Last data received 2 hours agoUpdated hourlyLIVE'

In [32]:
text = text.replace("WINDSPEED", "")
text = text.replace("km/hDIRECTION", ",")
text = text.replace("°WAVESHEIGHT", ",")
text = text.replace("mPERIOD", ",")
text = text.replace("sDIRECTION", ",")
text = text.replace("°Last data received ", ",")
text = text.replace(" ", ",")

In [33]:
text

'4.3,120,0.3,5,324,2,hours,agoUpdated,hourlyLIVE'

In [ ]:
columns = ["wspd", "wdir", "swvht", "tp", "wvdir"]
values = np.array(elements[2].text[0:5])

In [24]:
elements[2].text
        

'BUOY OBSERVATIONTEMP AT 1m25.0°CTEMP AT 6m24.6°CLast data received 42 min. agoUpdated hourlyLIVE'

In [20]:
soup.find_all(attrs={'class': 'MuiCard-root'})[4].text

'WINDSPEED34.7km/hDIRECTION56°WAVESHEIGHT1.3mPERIOD5sDIRECTION82°Valid 1 hour agoUpdated every 6 hoursSOFAR MODEL'

In [15]:
soup.find_all(attrs={'class': 'MuiCard-root'})[1].text

'SATELLITE OBSERVATIONSURFACE TEMP24.3°CHISTORICAL MAX25.4°CDEGREE HEATING WEEKS0.0SST ANOMALY-1.0°C0246810121416182025Last data received 1 day agoUpdated dailyNOAA'

In [40]:
text = a.soup.find_all(attrs={'class': 'MuiCard-root'})[1].text
text = text.replace('BUOY OBSERVATIONTEMP AT 1m', '')
text = text.split('°')[0]
text

'27.8'

In [26]:
text = x
text = text.replace('WINDSPEED', '')
text = text.replace('km/hDIRECTION', ',')
text = text.replace('°WAVESHEIGHT', ',')
text = text.replace('mPERIOD', ',')
text = text.replace('sDIRECTION', ',')
text = text.replace('°Last data received ', ',')
text = text.replace(' ',',')
text = text.split(',')[0:7]

In [28]:
x

'WINDSPEED5.8km/hDIRECTION302°WAVESHEIGHT0.8mPERIOD9sDIRECTION340°Last data received 59 min. agoUpdated hourlyLIVE'

In [11]:
url=f"{a.url}{a.stations.iloc[0].url}"
url

'https://aqualink.org/sites/1186'

In [12]:
url=f"{a.url}{a.stations.iloc[0].url}"
a.driver.get(url)
time.sleep(10)

In [13]:
soup=BeautifulSoup(a.driver.page_source, 'html.parser')


In [21]:
l = soup.find(attrs={'class': 'jss138'})

In [40]:
datetime.now().strftime("%Y-%m-%d %H:00:00")

'2021-11-25 21:00:00'

In [51]:
l = soup.find(attrs={'class': 'jss103'})
l.text[0:-2]

'28.0'

In [29]:
text = l.text
text

'WINDSPEED5.8km/hDIRECTION208°WAVESHEIGHT1.1mPERIOD10sDIRECTION162°Last data received 56 min. agoUpdated hourlyLIVE'

In [35]:
text = l.text
text = text.replace('WINDSPEED', '')
text = text.replace('km/hDIRECTION', ',')
text = text.replace('°WAVESHEIGHT', ',')
text = text.replace('mPERIOD', ',')
text = text.replace('sDIRECTION', ',')
text = text.replace('°Last data received ', ',')
text = text.replace(' ',',')
text = text.split(',')[0:7]

In [36]:
text[-1]

'min.'

In [ ]:
options = Options()
self.options = def_args_prefs(self.options, self.args, self.preferences)
self.driver = webdriver.Chrome(options=self.options)

self.db = GetData()
self.equip = equip
self.stations = self.db.get(table='stations', institution=['=', 'aqualink'], data_type=['=', self.equip])

self.url='https://aqualink.org/sites/'

In [34]:
p = PEBuoy()

In [35]:
p.get()

354.99750039428926
driver is running
chrome is still running, we can quit
driver has died


In [62]:
p.result

,date_time,wvdir,swvht,wvdir_sea,swvht_sea,wvdir_swell,swvht_swell,wspd,gust,wdir,station_id
0,2021-09-12 19:50:00,125,1.52,102,1.36,138,0.69,5.19,5.96,203,895
